In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import arrow
import pandas as pd

Everything is for Spain (ES)

In [3]:
# Installed Capacity
capacity_df = pd.read_csv("raw/Installed Capacity per Production Type_202201010000-202301010000.csv")

capacity_df.loc[:, "Grid Node"] = "ES"
capacity_df.loc[:, "datetime"] = "2022-01-01"
capacity_df = capacity_df.rename(columns={"Production Type": "Fuel Type", "2022 [MW]": "Capacity (MW)"})

capacity_df.to_csv("installed_capacity.csv", index=False)

In [24]:
# Generation
generation_df = pd.read_csv("raw/Actual Generation per Production Type_202211010000-202211020000.csv")

generation_df.loc[:, "Grid Node"] = "ES"
generation_df.loc[:, "unit"] = "MWh"

generation_df.loc[:, "datetime"] = generation_df.loc[:, "MTU"].apply(lambda x: arrow.get(x.split(" - ")[0], "DD.MM.YYYY HH:mm").format("YYYY-MM-DD HH:mm"))

generation_df = generation_df.drop(columns=["MTU", "Area"])
generation_df = generation_df.rename(columns={c: c.split("  - Actual Aggregated [MW]")[0] + " (MW)" for c in generation_df.columns if "Actual Aggregated [MW]" in c})
generation_df_out = generation_df.rename(columns={
    "Biomass (MW)": "BIOMASS",
    "Fossil Brown coal/Lignite (MW)": "BROWN_COAL",
    "Fossil Coal-derived gas (MW)": "COAL_DERIVED_GAS",
    "Fossil Gas (MW)": "OTHER_GAS",
    "Fossil Hard coal (MW)":"HARD_COAL",
    "Fossil Oil shale (MW)":"SHALE_OIL",
    "Fossil Oil (MW)":"OTHER_OIL",
    "Fossil Peat (MW)":"PEAT",
    "Geothermal (MW)":"GEOTHERMAL",
    "Hydro Pumped Storage (MW)":"HYDRO_PUMPED_STORAGE",
    "Hydro Pumped Storage  - Actual Consumption [MW]":"HYDRO_PUMPED_STORAGE_CONSUMPTION",
    "Hydro Run-of-river and poundage (MW)": "HYDRO_RUN_OF_RIVER",
    "Hydro Water Reservoir (MW)": "HYDRO_RESERVOIR",
    "Marine (MW)": "HYDRO",
    "Nuclear (MW)":"NUCLEAR",
    "Other (MW)": "OTHER",
    "Other renewable (MW)":"OTHER_RENEWABLE",
    "Solar (MW)":"SOLAR",
    "Waste (MW)":"WASTE",
    "Wind Offshore (MW)": "WIND_OFF_SHORE",
    "Wind Onshore (MW)":"WIND_ON_SHORE"
})

generation_df_out.fillna(0).to_csv("generation.csv", index=False)

In [5]:
# Imports and Exports
imports_exports_df = pd.read_csv("raw/Cross-Border Physical Flow_202211010000-202211020000.csv")

imports_exports_df.loc[:, "Grid Node"] = "ES"
imports_exports_df.loc[:, "datetime"] = imports_exports_df.loc[:, "Time (UTC)"].apply(lambda x: arrow.get(x.split(" - ")[0], "DD.MM.YYYY HH:mm").format("YYYY-MM-DD HH:mm"))

imports_exports_df = imports_exports_df.rename(columns={"BZN|FR > BZN|ES [MW]": "FR import (MW)", "BZN|ES > BZN|FR [MW]": "FR export (MW)"})
imports_exports_df = imports_exports_df.drop(columns=["Time (UTC)"])

imports_exports_df.to_csv("imports_exports.csv", index=False)

In [6]:
# Load forecast
load_forecast_df = pd.read_csv("raw/Total Load - Day Ahead _ Actual_202211010000-202211020000.csv")

load_forecast_df.loc[:, "Grid Node"] = "ES"
load_forecast_df.loc[:, "datetime"] = load_forecast_df.loc[:, "Time (UTC)"].apply(lambda x: arrow.get(x.split(" - ")[0], "DD.MM.YYYY HH:mm").format("YYYY-MM-DD HH:mm"))
load_forecast_df = load_forecast_df.drop(columns=["Time (UTC)"])
load_forecast_df = load_forecast_df.rename(columns={"Day-ahead Total Load Forecast [MW] - BZN|ES": "Demand Forecast (MW)"})

load_forecast_df.to_csv("load_forecast.csv", index=False)